Source code of every test for the task A with a BERT model

In [1]:
import numpy as np
import pandas as pd

# Load CSV files.
#CSV task A 
def getData():
    df_train_data = pd.read_csv("data/Training_Data/subtaskA_data_all.csv")
    df_train_answers = pd.read_csv("data/Training_Data/subtaskA_answers_all.csv")

    df_train = pd.merge(df_train_data,df_train_answers,on='id', how='left').drop(['id'], axis=1)
    
    df_dev_data = pd.read_csv("data/Dev_Data/subtaskA_dev_data.csv")
    df_dev_answers = pd.read_csv("data/Dev_Data/subtaskA_gold_answers.csv")

    df_dev = pd.merge(df_dev_data,df_dev_answers,on='id', how='left').drop(['id'], axis=1)

    df_test_data = pd.read_csv("data/Test_Data/subtaskA_test_data.csv")
    df_test_answers = pd.read_csv("data/Test_Data/subtaskA_gold_answers.csv")

    df_test= pd.merge(df_test_data,df_test_answers,on='id', how='left').drop(['id'], axis=1)
    
    return df_train, df_dev, df_test

df_train_A, df_dev_A, df_test_A = getData()



In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

Methods to pre-process the dataframe

In [3]:
def lemmatizer(text):
    """
    Receives a string as an input and lemmatizes it.
    """
    str = ""
    doc = nlp(text)
    for token in doc:
        str+=" "+token.lemma_
    return str 

def removeStopWords(text):
    """
    Receives a string and remove stop words from it.
    """
    str = ""
    doc = nlp(text)
    for token in doc:
        if(not token.is_stop):
            str+=" "+token.text
    return str 



In [4]:
def pre_process(df, function):
    newdf = df[['sent0', 'sent1']]
    newdf.loc[:,"sent0"] = df.sent0.apply(function)
    newdf.loc[:,"sent1"] = df.sent1.apply(function)
    return newdf

Process of data frame, create subsample of it

In [5]:
def subsampleData():
    # subsample data 
    train = df_train_A.sample(n=1000, random_state=42)
    X_train = train[['sent0', 'sent1']]
    y_train = train['answer']

    return X_train, y_train

# use the dev set for testing
X_test = df_dev_A[['sent0', 'sent1']]
y_test = df_dev_A['answer']

Importation of the BERT model

In [6]:
from transformers import BertModel
from bert_sklearn import BertClassifier

In [7]:
model = BertClassifier(max_seq_length=64, train_batch_size=16)
#model.num_mlp_layers = 3
model.max_seq_length = 64
model.epochs = 3
#model.learning_rate = 4e-5
                             
model

Building sklearn text classifier...


BertClassifier(max_seq_length=64, train_batch_size=16)

In [8]:
X_train_sample, y_train_sample = subsampleData()

Fit with different preprocess type                                                                

In [9]:
X_train_sample.head()

,sent0,sent1
6252,a duck walks on three legs,a duck walks on two legs
4684,Jack's mom praised him because he broke the plate,Jack's mom condemned him because he broke the ...
1731,People use electricity to buy things,People use money to buy things
4742,"The speaker is damaged, thus I can't hear anyt...","The display is damaged, thus I can't hear anyt..."
4521,Santa Claus is the legend of the East,Santa Claus is the legend of the West


In [ ]:
model_classic = model.fit(X_train_sample, y_train_sample)

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Training  :   0%|                                                                               | 0/57 [00:00<?, ?it/s]C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\bert_sklearn\model\pytorch_pretrained\optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:1485.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
Training  :  32%|██████████████████▎                                       | 18/57 [02:48<04:45,  7.33s/it, loss=0.798]

With only lemma

In [ ]:
X_train = pre_process(X_train_sample,lemmatizer)
X_train.head()

In [ ]:
model_lemma = model.fit(X_train, y_train)

Remove stop words

In [ ]:
X_train = pre_process(X_train_sample,removeStopWords)
X_train.head()

In [ ]:
model_stopWords = model.fit(X_train, y_train)

Score of models

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [ ]:
def test_performance(model, x_test, y_test):
    y_pred = model.predict(x_test)
    print(classification_report(y_pred=y_pred, y_true=y_test))
    return f1_score(y_pred=y_pred, y_true=y_test, average="macro"), f1_score(y_pred=y_pred, y_true=y_test, average="micro")

In [ ]:
f1micro, f1macro = test_performance(model_classic, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")

In [ ]:
f1micro, f1macro = test_performance(model_lemma, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")

In [ ]:
f1micro, f1macro = test_performance(model_stopWords, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")

To save a model

In [ ]:
#save model to disk
savefile = 'BERT_TaskA.bin'
model.save(savefile)